In [1]:
#Mehmet Cagri Aksoy . 2024
#github.com/mcagriaksoy

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!unzip /content/drive/MyDrive/Datasets/Pollen/15March_Version_Pollen_Dataset.zip

Streaming output truncated to the last 5000 lines.
 extracting: train/74_qualea_multiflora/aug_0_5954.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_600.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6077.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6088.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6145.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6150.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6196.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6237.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6244.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6384.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6485.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6509.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6595.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6612.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6613.jpeg  
 extracting: train/74_qualea_multiflora/aug_0_6648.jpeg  
 extracting: train/74_

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os

In [6]:
# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((50, 50)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((50, 50)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [9]:
# Load dataset
data_dir = '/content/train'
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

In [12]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [13]:
# Create DataLoaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [14]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Modify the final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 117)  # Adjusted to 117 classes


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 74.9MB/s]


In [15]:
# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [16]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [17]:
# Training loop with model saving
num_epochs = 25
best_model_wts = model.state_dict()
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Deep copy the model if it has the best accuracy so far
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()

# Load best model weights
model.load_state_dict(best_model_wts)

# Save the best model weights
torch.save(model.state_dict(), '/content/drive/MyDrive/best_model_weights.pth')

print('Training complete')
print(f'Best val Acc: {best_acc:.4f}')

Epoch 0/24
----------
train Loss: 1.9315 Acc: 0.4633
val Loss: 1.0375 Acc: 0.6619
Epoch 1/24
----------
train Loss: 0.7592 Acc: 0.7508
val Loss: 0.7436 Acc: 0.7747
Epoch 2/24
----------
train Loss: 0.4951 Acc: 0.8367
val Loss: 0.5307 Acc: 0.8316
Epoch 3/24
----------
train Loss: 0.3581 Acc: 0.8776
val Loss: 0.5329 Acc: 0.8319
Epoch 4/24
----------
train Loss: 0.2826 Acc: 0.9022
val Loss: 0.4540 Acc: 0.8546
Epoch 5/24
----------
train Loss: 0.2263 Acc: 0.9209
val Loss: 0.5013 Acc: 0.8549
Epoch 6/24
----------
train Loss: 0.1973 Acc: 0.9333
val Loss: 0.3214 Acc: 0.8955
Epoch 7/24
----------
train Loss: 0.1663 Acc: 0.9408
val Loss: 0.3653 Acc: 0.8823
Epoch 8/24
----------
train Loss: 0.1590 Acc: 0.9437
val Loss: 0.4587 Acc: 0.8630
Epoch 9/24
----------
train Loss: 0.1430 Acc: 0.9492
val Loss: 0.3954 Acc: 0.8813
Epoch 10/24
----------
train Loss: 0.1129 Acc: 0.9614
val Loss: 0.3693 Acc: 0.8913
Epoch 11/24
----------
train Loss: 0.1260 Acc: 0.9556
val Loss: 0.3568 Acc: 0.8990
Epoch 12/24
--

In [24]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 74.3 MB/s eta 0:00:00


In [27]:

# Save the model as ONNX
dummy_input = torch.randn(1, 3, 50, 50).to(device)
onnx_path = '/content/drive/MyDrive/Models/best_model.onnx'
torch.onnx.export(model, dummy_input, onnx_path,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

print('Training complete')
print(f'Best val Acc: {best_acc:.4f}')
print(f'Model saved as ONNX at {onnx_path}')

Training complete
Best val Acc: 0.9183
Model saved as ONNX at /content/drive/MyDrive/Models/best_model.onnx
